In [3]:
import importlib
import sub
import subprocess as sp
importlib.reload(sub) 
from pathlib import Path
from tqdm import tqdm
import os
import sys
import contextlib
from contextlib import contextmanager
import nibabel as nib
import numpy as np
import math

@contextmanager
def suppress_stdout_stderr():
    """Temporarily redirect stdout and stderr to os.devnull."""
    devnull = open(os.devnull, 'w')
    old_stdout, old_stderr = sys.stdout, sys.stderr
    try:
        sys.stdout = devnull
        sys.stderr = devnull
        yield
    finally:
        sys.stdout = old_stdout
        sys.stderr = old_stderr
        devnull.close()


In [2]:
full_sub = {}
cwd = Path.cwd()
sub_folder = cwd / "derivates"
for folder_name in sub_folder.iterdir():
    if folder_name.is_dir() and folder_name.name.startswith("sub_"):
        if folder_name.name == "sub_025":  # REMOVE SUB_25 he has no hypo data
            continue
        subject_name = folder_name.name
        full_sub[subject_name] = sub.Subject(subject_name)
        

In [5]:
def funct_motion_correction():
    for key, sub in tqdm(full_sub.items()):
            print(f"SUBJECT {key}")
            hyper = sub.data_hyper["func"]
            hypo = sub.data_hypo["func"]
            condition = {"hyper": hyper, "hypo": hypo}
            for cond in condition:
                for run in [f"R{i}" for i in range(1,5)]:
                    print(f"Processing {key} {cond} for {run}")
                    func = condition[cond][run]
                    func.reorient()
                    func.motion_correction()
                print("|")
            print("-------------------------")
            
def func_epi_to_t1(sub_ids):
    if type(sub_ids) is str:
        sub_ids = [sub_ids]
    selected_sub = {key: full_sub[key] for key in sub_ids}
    for key, sub in tqdm(selected_sub.items()):
            hyper = sub.data_hyper["func"]
            hypo = sub.data_hypo["func"]
            condition = {"hyper": hyper, "hypo": hypo}
            for cond in condition:
                for run in [f"R{i}" for i in range(1,5)]:
                    print(f"Processing {key} {cond} for {run}")
                    func = condition[cond][run]
                    with suppress_stdout_stderr():
                        func.func_to_t1()
                print("|")
            print("-------------------------")


def func_t1_mask_to_epi(sub_ids):
    if type(sub_ids) is str:
        sub_ids = [sub_ids]
    selected_sub = {key: full_sub[key] for key in sub_ids}
    for key, sub in tqdm(selected_sub.items()):
            hyper = sub.data_hyper["func"]
            hypo = sub.data_hypo["func"]
            condition = {"hyper": hyper, "hypo": hypo}
            for cond in condition:
                for run in [f"R{i}" for i in range(1,5)]:
                    print(f"Processing {key} {cond} for {run}")
                    func = condition[cond][run]
                    with suppress_stdout_stderr():
                        func.t1_mask_to_epi()
                print("|")
            print("-------------------------")

def func_nuissance_regression(sub_ids):
    if type(sub_ids) is str:
        sub_ids = [sub_ids]
    selected_sub = {key: full_sub[key] for key in sub_ids}
    for key, sub in tqdm(selected_sub.items()):
            hyper = sub.data_hyper["func"]
            hypo = sub.data_hypo["func"]
            condition = {"hyper": hyper, "hypo": hypo}
            for cond in condition:
                for run in [f"R{i}" for i in range(1,5)]:
                    print(f"Processing {key} {cond} for {run}")
                    func = condition[cond][run]
                    with suppress_stdout_stderr():
                        func.nuissance_regression(plot_design = False)
                print("|")
            print("-------------------------")
            

def func_epi_to_mni(sub_ids):
    if type(sub_ids) is str:
        sub_ids = [sub_ids]
    selected_sub = {key: full_sub[key] for key in sub_ids}
    for key, sub in tqdm(selected_sub.items()):
            hyper = sub.data_hyper["func"]
            hypo = sub.data_hypo["func"]
            condition = {"hyper": hyper, "hypo": hypo}
            for cond in condition:
                for run in [f"R{i}" for i in range(1,5)]:
                    print(f"Processing {key} {cond} for {run}")
                    func = condition[cond][run]
                    with suppress_stdout_stderr():
                        func.epi_to_mni()
                print("|")
            print("-------------------------")
            
def anat_gm_mask_to_mni(sub_ids, th = 0.2):
    if type(sub_ids) is str:
        sub_ids = [sub_ids]
    selected_sub = {key: full_sub[key] for key in sub_ids}
    for key, sub in tqdm(selected_sub.items()):
            hyper = sub.data_hyper["anat"]
            hypo = sub.data_hypo["anat"]
            condition = {"hyper": hyper, "hypo": hypo}
            for cond in condition:
                #print(f"Processing {key} {cond} anat")
                anat = condition[cond]
                with suppress_stdout_stderr():
                    anat.gm_mask_to_mni()
                    anat.threshold_gm_mask(th=th)
            #print("-------------------------")


def func_smoothing(sub_ids):
    if type(sub_ids) is str:
        sub_ids = [sub_ids]
    selected_sub = {key: full_sub[key] for key in sub_ids}
    for key, sub in tqdm(selected_sub.items()):
            hyper = sub.data_hyper["func"]
            hypo = sub.data_hypo["func"]
            condition = {"hyper": hyper, "hypo": hypo}
            for cond in condition:
                for run in [f"R{i}" for i in range(1,5)]:
                    print(f"Processing {key} {cond} for {run}")
                    func = condition[cond][run]
                    with suppress_stdout_stderr():
                        func.smoothing(fwhm=6)
                print("|")
            print("-------------------------")

In [6]:
anat_gm_mask_to_mni(full_sub, th=0.2)

100%|██████████| 17/17 [01:19<00:00,  4.65s/it]


In [7]:


def make_group_mask(subjs_ids, out_path= None, frac=0.6):
    if out_path is None:
        out_path = Path.cwd() / "derivates" / f"group_mask_gm_th{frac*100:.0f}.nii.gz"
    subj_mask_paths = []
    for sub in subjs_ids:
        sub_gm_mask_hyper = full_sub[sub].data_hyper["anat"].t1_gm_mask_thr
        sub_gm_mask_hypo = full_sub[sub].data_hypo["anat"].t1_gm_mask_thr
        subj_mask_paths.append(sub_gm_mask_hyper)
        subj_mask_paths.append(sub_gm_mask_hypo)
    N = len(subj_mask_paths)
    print(f"Creating group mask from {N} subjects (threshold = {frac*100:.0f}%)")

    # Charger le premier masque comme référence
    ref_img = nib.load(subj_mask_paths[0])
    acc = np.zeros(ref_img.shape, dtype=np.int32)

    # Additionner tous les masques binaires
    for p in subj_mask_paths:
        acc += (nib.load(p).get_fdata() > 0).astype(np.int32)

    # Calcul du seuil (arrondi vers le haut)
    thr = math.ceil(frac * N)

    # Seuil et binarisation
    grp = (acc >= thr).astype(np.uint8)

    # Sauvegarde avec les mêmes affine/header
    grp_img = nib.Nifti1Image(grp, ref_img.affine, ref_img.header)
    grp_img.set_data_dtype(np.uint8)
    nib.save(grp_img, out_path)

    print(f"[OK] Group mask saved to {out_path} (voxels kept = {int(grp.sum())})")



In [9]:
make_group_mask(full_sub, frac=0.5)

Creating group mask from 34 subjects (threshold = 50%)
[OK] Group mask saved to /Volumes/T7/MAP/derivates/group_mask_gm_th50.nii.gz (voxels kept = 147374)


In [6]:
# for id, sub in full_sub.items():
#     hyper = sub.data_hyper["func"]
#     hypo = sub.data_hypo["func"]
#     runs = [f"R{i}" for i in range(1,5)]
#     for run in runs:
#         # hyper_func = hyper[run].func_img_or_mc_bet_clean_withGSR_mni
#         # hypo_func = hypo[run].func_img_or_mc_bet_clean_withGSR_mni
#         hyper_func = hyper[run].func_img_or
#         hypo_func = hypo[run].func_img_or
#         hyper_img = nib.load(hyper_func)
#         hypo_img = nib.load(hypo_func)
#         hyper_data = hyper_img.get_fdata()
#         hypo_data = hypo_img.get_fdata()
#         print(f"{id} {run} hyper: {hyper_data.shape} hypo: {hypo_data.shape}")

In [7]:
# folder = cwd / "derivates"
# for sub in folder.iterdir():
#     if "sub" not in str(sub.name):
#         continue
#     if "sub_" in sub.name:
#         func = sub / "func"
#         func_hyper = func / "hyper"
#         func_hypo = func / "hypo"
#     for run in func_hyper.iterdir():
#         no_gsr = 0 
#         gsr = 0
#         for run_file in run.iterdir():
#             if "clean_nogsr_mni_smooth" in run_file.name:
#                 no_gsr += 1
#             if "clean_withgsr_mni_smooth" in run_file.name:
#                 gsr += 1
#         if gsr + no_gsr < 2:
#             print(f"Missing files in {run_file.parent}")
#         else :
#             print(f"Subject {sub.name} {run.name} hyper complete")
#     for run in func_hypo.iterdir():
#         no_gsr = 0 
#         gsr = 0
#         for run_file in run.iterdir():
#             if "clean_nogsr" in run_file.name:
#                 no_gsr += 1
#             if "clean_withgsr" in run_file.name:
#                 gsr += 1
#         if gsr + no_gsr < 2:
#             print(f"Missing files in {run_file.parent}")
#         else :
#             print(f"Subject {sub.name} {run.name} hypo complete")


In [11]:

def split_fmri_4d_to_3d(input_file, output_dir):
    input_file = Path(input_file)
    output_dir = Path(output_dir)
    output_prefix = output_dir / Path(input_file.stem).stem  
    cmd = ["fslsplit", str(input_file), str(output_prefix) + "_", "-t"]
    sp.run(cmd, check=True)


def nii_gz_to_nii(input_file, output_dir = None):
    input_file = Path(input_file)
    if output_dir is None:
        output_dir = input_file.parent
    stem = Path(input_file.stem).stem       
    output_name = f"sw_{stem}.nii"
    output_file = output_dir / output_name
    cmd = ["fslchfiletype", "NIFTI", str(input_file), str(output_file)]
    sp.run(cmd, check=True)
    return output_file



def format_cap_analysis(sub_ids, gsr = True):
    if type(sub_ids) is str:
        sub_ids = [sub_ids]
    selected_sub = {key: full_sub[key] for key in sub_ids}
    for id, sub  in tqdm(selected_sub.items()):
        for cond in ["hyper", "hypo"]:
            for run in [f"R{i}" for i in range(1,5)]:
                par_file = sub.data_hyper["func"][run].func_img_or_mc_par if cond == "hyper" else sub.data_hypo["func"][run].func_img_or_mc_par
                if gsr : 
                    input_file = sub.data_hyper["func"][run].func_img_or_mc_bet_clean_withGSR_mni_smooth if cond == "hyper" else sub.data_hypo["func"][run].func_img_or_mc_bet_clean_withGSR_mni_smooth
                    output_dir = Path.cwd() / "cap_analysis_withgsr" / f"{id}_{cond}_{run}" / "func"
  
                    #split_fmri_4d_to_3d(input_file, output_dir)
                else :
                    input_file = sub.data_hyper["func"][run].func_img_or_mc_bet_clean_withoutGSR_mni_smooth if cond == "hyper" else sub.data_hypo["func"][run].func_img_or_mc_bet_clean_withoutGSR_mni_smooth
                    output_dir = Path.cwd() / "cap_analysis_nogsr" / f"{id}_{cond}_{run}" / "func"
     
                    #split_fmri_4d_to_3d(input_file, output_dir)
                arr = np.loadtxt(par_file)        # Rx Ry Rz Tx Ty Tz
                arr_cap = np.c_[arr[:, 3:6], arr[:, 0:3]]   # Tx Ty Tz Rx Ry Rz
                np.savetxt(output_dir / "motion_for_CAP.txt", arr_cap, fmt="%.7g")
format_cap_analysis(full_sub, gsr = True)





100%|██████████| 17/17 [00:00<00:00, 135.01it/s]


In [19]:
folder_sub1 = []
for folder in Path("/Volumes/T7/MAP/cap_analysis_withgsr").iterdir():
    if "sub_001" in str(folder.name):
        folder_sub1.append(Path(folder))

for folder in folder_sub1:
    func = folder / "func"
    for file in tqdm(func.iterdir()):
        if "sub" not in str(file.name):
            continue
        nii_gz_to_nii(file)
    

392it [01:27,  4.46it/s]
197it [00:54,  3.60it/s]
197it [00:56,  3.48it/s]
197it [00:57,  3.42it/s]
197it [00:56,  3.49it/s]
197it [00:54,  3.59it/s]
197it [00:55,  3.56it/s]
197it [00:54,  3.60it/s]


In [20]:
def apply_mask_to_nii(input_file, mask_file, output_file = None):
    if output_file is None:
        # we replace the inoput by the masked version
        input_path = Path(input_file)
        output_file = input_path
    input_img = nib.load(input_file)
    mask_img = nib.load(mask_file)
    
    input_data = input_img.get_fdata()
    mask_data = mask_img.get_fdata()
    
    if input_data.shape != mask_data.shape:
        raise ValueError("Input image and mask must have the same shape.")
    
    masked_data = input_data * mask_data
    
    masked_img = nib.Nifti1Image(masked_data, input_img.affine, input_img.header)
    nib.save(masked_img, output_file)
    


In [22]:
mask = "/Volumes/T7/MAP/derivates/group_mask_gm_th60.nii.gz"
folder_sub1 = []
for folder in Path("/Volumes/T7/MAP/cap_analysis_withgsr").iterdir():
    if "sub_001" in str(folder.name):
        folder_sub1.append(Path(folder))

for folder in folder_sub1:
    func = folder / "func"
    for file in tqdm(func.iterdir()):
        if "sub" not in str(file.name):
            continue
        apply_mask_to_nii(file,mask)

392it [00:12, 32.22it/s]
392it [00:13, 28.16it/s]
392it [00:13, 28.98it/s]
392it [00:13, 28.46it/s]
392it [00:13, 28.44it/s]
392it [00:13, 28.60it/s]
392it [00:13, 29.07it/s]
392it [00:13, 29.43it/s]


In [32]:

uno = nib.load("/Volumes/T7/MAP/cap_analysis_withgsr/sub_001_hyper_R2/func/sw_sub_001_HYPER_runR2_mc_bet_clean_withgsr_mni_smooth_0000.nii")

duo = nib.load("/Volumes/T7/MAP/cap_analysis_withgsr/sub_001_hyper_R1/func/sw_sub_001_HYPER_runR1_mc_bet_clean_withgsr_mni_smooth_0000.nii")
duo.get_fdata().shape, uno.get_fdata().shape

((91, 109, 91), (91, 109, 91))

In [31]:
MNI_HEAD_2mm = Path(os.environ.get("FSLDIR", "/usr/local/fsl") + "/data/standard/MNI152_T1_2mm.nii.gz")
nib.load(MNI_HEAD_2mm).get_fdata().shape

(91, 109, 91)

In [3]:
import subprocess
from pathlib import Path

cwd = Path.cwd()
# Input and output paths
func_mc = Path("/Volumes/T7/MAP/derivates/sub_002/func/hyper/runR1/mc/sub_002_HYPER_runR1_mc.nii.gz")


# Output filenames (will match FSL's convention)
conf_file =  cwd / "conf_FD.txt"
plot_file = cwd/ "fd_plot2.png"

# Threshold (Power et al. FD > 0.5 mm is typical)
threshold = 0.5

# Build the command
cmd = [
    "fsl_motion_outliers",
    "-i", str(func_mc),
    "-o", str(conf_file),
    "--fd",
    f"--thresh={threshold}",
    "-p", str(plot_file)
]

# Run it
subprocess.run(cmd, check=True)
print("✅ FD computed and saved to:", conf_file)


✅ FD computed and saved to: /Volumes/T7/MAP/conf_FD.txt


In [9]:
list(Path("images/sub_017/anat/hypo").glob("*.nii.gz"))[0]

PosixPath('images/sub_017/anat/hypo/sub_017_HYPO_T1w.nii.gz')

In [ ]:
# def run_clustering(frames_buffer, k=8, random_state=42):
#     print(f"--- Running Clustering (k={k}) ---")
    
#     # 1. Stack
#     X_raw = np.vstack(frames_buffer)
#     print(f"Data shape: {X_raw.shape}")
    
#     # --- CORRECTION FOR EXACT PAPER REPLICATION ---
#     # The paper uses "1 - Pearson's Correlation".
#     # Pearson requires Centering (Mean Removal) + Normalization.
#     # Your previous code only did Normalization (Cosine).
    
#     # Step A: Subtract the Spatial Mean (Row-wise centering)
#     X_centered = X_raw - X_raw.mean(axis=1, keepdims=True)
    
#     # Step B: L2 Normalization
#     # Now, dot product of these vectors == Pearson Correlation
#     X_norm = normalize(X_centered, axis=1, norm='l2')
#     # ----------------------------------------------
    
#     # 3. K-Means
#     kmeans = KMeans(n_clusters=k, random_state=random_state, n_init=10)
#     labels = kmeans.fit_predict(X_norm)
    
#     # Return X_raw (not centered) for map reconstruction, so maps show true intensity
#     return labels, X_raw

In [ ]:

# def find_optimal_k(frames_buffer, k_range=range(2, 21), random_state=42):
#     """
#     Runs K-Means for a range of k values to help find the optimal number of clusters.
#     Returns a DataFrame of metrics and the normalized data for final clustering.
#     """
#     print("--- Preprocessing Data (Once) ---")
    
#     # 1. Stack
#     X_raw = np.vstack(frames_buffer)
    
#     # 2. Centering (Spatial Mean Removal) - Crucial for Pearson Correlation
#     X_centered = X_raw - X_raw.mean(axis=1, keepdims=True)
    
#     # 3. L2 Normalization - Crucial for Cosine/Pearson equivalence
#     X_norm = normalize(X_centered, axis=1, norm='l2')
    
#     results = []
    
#     print(f"--- Scanning k from {k_range.start} to {k_range.stop - 1} ---")
    
#     for k in tqdm(k_range, desc="K-Means k values", unit="k"):
#         # Run K-Means
#         kmeans = KMeans(n_clusters=k, random_state=random_state, n_init=10)
#         labels = kmeans.fit_predict(X_norm)
        
#         # Metric A: Inertia (Sum of squared distances) -> For Elbow Method
#         inertia = kmeans.inertia_
        
#         # Metric B: Silhouette Score (How distinct clusters are)
#         # Note: This is computationally expensive. If >10k frames, consider subsampling.
#         sil = silhouette_score(X_norm, labels)
        
#         results.append({"k": k, "inertia": inertia, "silhouette": sil})
#         print(f"k={k}: Inertia={inertia:.2f}, Silhouette={sil:.4f}")
        
#     df_results = pd.DataFrame(results)
    
#     # --- Plotting the Results ---
#     fig, ax1 = plt.subplots(figsize=(10, 6))

#     color = 'tab:blue'
#     ax1.set_xlabel('Number of Clusters (k)')
#     ax1.set_ylabel('Inertia (Lower is better)', color=color)
#     ax1.plot(df_results['k'], df_results['inertia'], color=color, marker='o')
#     ax1.tick_params(axis='y', labelcolor=color)

#     ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
#     color = 'tab:red'
#     ax2.set_ylabel('Silhouette Score (Higher is better)', color=color)
#     ax2.plot(df_results['k'], df_results['silhouette'], color=color, marker='x', linestyle='--')
#     ax2.tick_params(axis='y', labelcolor=color)

#     plt.title("Optimization of K: Elbow Method vs Silhouette")
#     plt.grid(True)
#     plt.show()
    
#     return df_results, X_norm, X_raw

# # Usage Example:
# # metrics_df, X_norm_ready, X_raw_ready = find_optimal_k(selected_frames_buffer)
# # Look at the plot, choose your best K (e.g., 8), then run your final clustering:
# # final_labels = KMeans(n_clusters=8, ...).fit_predict(X_norm_ready)